# 6장 데이터베이스 사용하기

1. 몽고디비 시작하기
2. 익스프레스에서 몽고디비 사용하기
3. 몽구스로 데이터베이스 다루기
4. 인덱스와 메소드 사용하기
5. 비밀번호 암호화하여 사용하기
6. MySQL 데이터베이스 사용하기(생략)

* Path 시스템 변수에 추가 후 실행
* 윈도우 사용자 계정 폴더 아래에 database/local 폴더 생성
* 명령프롬프트 창에서 mongod 명령어로 실행

In [4]:
import requests
from io import BytesIO
from PIL import Image

    C:\Users\WhiteHat>mongod --dbpath C:\Users\WhiteHat\database\local
```
...
2020-06-13T04:30:58.884+0900 I  NETWORK  [listener] Listening on 127.0.0.1
2020-06-13T04:30:58.885+0900 I  NETWORK  [listener] waiting for connections on port 27017
```

```
C:\Users\WhiteHat>mongo
MongoDB shell version v4.2.6
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("be4d165a-69db-418a-a8f4-5785df80b0ee") }
MongoDB server version: 4.2.6
Welcome to the MongoDB shell.
For interactive help, type "help".
For more comprehensive documentation, see
        http://docs.mongodb.org/
Questions? Try the support group
        http://groups.google.com/group/mongodb-user
Server has startup warnings:
2020-06-12T10:03:19.534+0900 I  CONTROL  [initandlisten]
2020-06-12T10:03:19.534+0900 I  CONTROL  [initandlisten] ** WARNING: Access control is not enabled for the database.
2020-06-12T10:03:19.534+0900 I  CONTROL  [initandlisten] **          Read and write access to data and configuration is unrestricted.
2020-06-12T10:03:19.534+0900 I  CONTROL  [initandlisten]
---
Enable MongoDB's free cloud-based monitoring service, which will then receive and display
metrics about your deployment (disk utilization, CPU, operation statistics, etc).

The monitoring data will be available on a MongoDB website with a unique URL accessible to you
and anyone you share the URL with. MongoDB may use this information to make product
improvements and to suggest MongoDB products and deployment options to you.

To enable free monitoring, run the following command: db.enableFreeMonitoring()
To permanently disable this reminder, run the following command: db.disableFreeMonitoring()
---

> use local
switched to db local
> db.users.insert({name:'donggyu',age:25})
WriteResult({ "nInserted" : 1 })
> db.users.find().pretty()
{
        "_id" : ObjectId("5ee3d8e42d9504750be80d18"),
        "name" : "donggyu",
        "age" : 25
}
> use local
switched to db local
> db.users.find().pretty()
{
        "_id" : ObjectId("5ee3d8e42d9504750be80d18"),
        "name" : "donggyu",
        "age" : 25
}
> db.users.remove({name:/dong/})
WriteResult({ "nRemoved" : 1 })
> db.users.find().pretty()
> db.users.insert({id:'test01',name:'donggyu',password:'123456'})
WriteResult({ "nInserted" : 1 })
> db.users.find().pretty()
{
        "_id" : ObjectId("5ee3ddb60b7822ebd5400bc9"),
        "id" : "test01",
        "name" : "donggyu",
        "password" : "123456"
}
>
```

## 사용자 조회 기능 만들기(authUser 함수 추가)

In [9]:
%%writefile ./src/app7.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// mongodb 모듈 사용
var MongoClient = require('mongodb').MongoClient;

var database;

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017';
    
    MongoClient.connect(databaseUrl, function(err, db) {
        if(err){
            console.log('데이터베이스 연결 시 에러 발생함.');
            return;
        }
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        // MongoDB 3 부터 요롷콤 바뀜.
        database = db.db('local');
    });
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    var users = database.collection('users');
    
    // toArray함수는 배열로 바꿔주는 함수임.
    // docs는 하나의 레코드와 같다.
    users.find({"id":id, "password":password}).toArray(function(err,docs){
        if(err){
            callback(err, null);
            return;
        }
        if(docs.length > 0){
            console.log('일치하는 사용자를 찾음.');
            callback(null, docs);
        } else {
            console.log('일치하는 사용자를 찾지 못함.');
            callback(null, null);
        }
    });
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Overwriting ./src/app7.js


In [10]:
URL = 'http://localhost:3000/process/login'
#response = requests.get(URL)
data = {'id': 'test01','password':'123456'}
res = requests.post(URL, data=data)
res.text

'<h1>사용자 로그인 성공</h1><div><p>사용자 : donggyu</p></div><br><br><a href="/public/login.html">다시 로그인하기</a>'

```
D:\폴더\학교\졸프\evote-project\WebServer\Study\src>node app7
익스프레스로 웹 서버를 실행함 : 3000
(node:21192) DeprecationWarning: current Server Discovery and Monitoring engine is deprecated, and will be removed in a future version. To use the new Server Discover and Monitoring engine, pass option { useUnifiedTopology: true } to the MongoClient constructor.
데이터베이스에 연결됨 : mongodb://localhost:27017
/process/login 라우팅 함수 호출됨.
요청 파라미터 : test01, 123456
authUser 호출됨 : test01, 123456
일치하는 사용자를 찾음.
[ { _id: ObjectID { _bsontype: 'ObjectID', id: [Buffer] },
    id: 'test01',
    name: 'donggyu',
    password: '123456' } ]
```

## 사용자 추가 기능 만들기(addUser 함수 추가)

In [21]:
%%writefile ./src/app7_1.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// mongodb 모듈 사용
var MongoClient = require('mongodb').MongoClient;

var database;

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017';
    
    MongoClient.connect(databaseUrl, function(err, db) {
        if(err){
            console.log('데이터베이스 연결 시 에러 발생함.');
            return;
        }
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        // MongoDB 3 부터 요롷콤 바뀜.
        database = db.db('local');
    });
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/adduser').post(function(req,res){
    console.log('/proccess/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
})

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    var users = database.collection('users');
    
    // toArray함수는 배열로 바꿔주는 함수임.
    // docs는 하나의 레코드와 같다.
    users.find({"id":id, "password":password}).toArray(function(err,docs){
        if(err){
            callback(err, null);
            return;
        }
        if(docs.length > 0){
            console.log('일치하는 사용자를 찾음.');
            callback(null, docs);
        } else {
            console.log('일치하는 사용자를 찾지 못함.');
            callback(null, null);
        }
    });
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    var users = database.collection('users');
    users.insertMany([{"id":id,"password":password,"name":name}], function(err, result){
        if (err) throw err;
        if(result.insertedCount > 0){
            console.log('사용자 추가됨 : ' + result.insertedCount);
            callback(null, result);
        }else{
            console.log('추가된 레코드가 없음.');
            callback(null, null);
        }
    })
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Overwriting ./src/app7_1.js


In [13]:
%%writefile ./src/public/adduser.html
<!DOCTYPE html>
<html>
    <head>
        <meta charset="utf-8">
        <title>사용자 추가</title>
    </head>
    <body>
        <h1>사용자 추가</h1>
        <br>
        <form method="post" action="/process/adduser">
            <table>
                <tr>
                    <td><label>아이디</label></td>
                    <td><input type="text" name="id"></td>
                </tr>
                <tr>
                    <td><label>비밀번호</label></td>
                    <td><input type="password" name="password"></td>
                </tr>
                <tr>
                    <td><label>사용자이름</label></td>
                    <td><input type="text" name="name"></td>
                </tr>
            </table>
            <input type="submit" value="전송" name="">
        </form>
    </body>
</html>

Overwriting ./src/public/adduser.html


In [22]:
URL = 'http://localhost:3000/process/adduser'
#response = requests.get(URL)
data = {'id': 'test02','password':'123456','name':'donggyu'}
res = requests.post(URL, data=data)
res.text

'<h1>사용자 추가 성공</h1><div><p>사용자 : donggyu</p></div><br><br><a href="/public/login.html">다시 로그인하기</a>'

```
D:\폴더\학교\졸프\evote-project\WebServer\Study\src>node app7_1.js
익스프레스로 웹 서버를 실행함 : 3000
(node:4056) DeprecationWarning: current Server Discovery and Monitoring engine is deprecated, and will be removed in a future version. To use the new Server Discover and Monitoring engine, pass option { useUnifiedTopology: true } to the MongoClient constructor.
데이터베이스에 연결됨 : mongodb://localhost:27017
/proccess/adduser 라우팅 함수 호출됨.
요청 파라미터 : test02, 123456, donggyu
addUuser 호출됨 : test02, 123456, donggyu
{ result: { ok: 1, n: 1 },
  ops:
   [ { id: 'test02',
       password: '123456',
       name: 'donggyu',
       _id: [ObjectID] } ],
  insertedCount: 1,
  insertedIds: { '0': ObjectID { _bsontype: 'ObjectID', id: [Buffer] } } }
사용자 추가됨 : 1
```

# 몽구스로 데이터베이스 다루기

* 컬렉션 안에 같은 자료형의 객체를 넣는 방법 제공(관계형 데이터베이스에 익숙한 사람들을 위해 제공)

```
    var mongoose = require('mongoose');
    
D:\폴더\학교\졸프\evote-project\WebServer\Study\src>npm install mongoose --save
npm WARN saveError ENOENT: no such file or directory, open 'D:\폴더\학교\졸프\evote-project\WebServer\Study\src\package.json'
npm WARN enoent ENOENT: no such file or directory, open 'D:\폴더\학교\졸프\evote-project\WebServer\Study\src\package.json'
npm WARN src No description
npm WARN src No repository field.
npm WARN src No README data
npm WARN src No license field.

+ mongoose@5.9.18
added 12 packages from 8 contributors and audited 162 packages in 1.518s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities
```

In [34]:
%%writefile ./src/app8.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;
var UserSchema;
var UserModel

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017/local';
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        
            
        UserSchema = mongoose.Schema({
            id: String,
            name: String,
            password: String
        });
        console.log('UserSchema 정의함.');
        
        UserModel = mongoose.model('users', UserSchema);
        console.log('UserModel 정의함.');
        
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/adduser').post(function(req,res){
    console.log('/proccess/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
})

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    UserModel.find({"id":id, "password":password}, function(err, docs){
        if(err){
            callback(err, null);
            return;
        }    
        if(docs.length > 0){
            console.log('일치하는 사용자를 찾음.');
            callback(null, docs);
        }else{
            console.log('일치하는 사용자를 찾지 못함.');
            callback(null, null);
        }  
    });
    
    /*var users = database.collection('users');
    // toArray함수는 배열로 바꿔주는 함수임.
    // docs는 하나의 레코드와 같다.
    users.find({"id":id, "password":password}).toArray(function(err,docs){
        if(err){
            callback(err, null);
            return;
        }
        if(docs.length > 0){
            console.log('일치하는 사용자를 찾음.');
            callback(null, docs);
        } else {
            console.log('일치하는 사용자를 찾지 못함.');
            callback(null, null);
        }
    });*/
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
    
    /*var users = database.collection('users');
    users.insertMany([{"id":id,"password":password,"name":name}], function(err, result){
        if (err) throw err;
        if(result.insertedCount > 0){
            console.log('사용자 추가됨 : ' + result.insertedCount);
            callback(null, result);
        }else{
            console.log('추가된 레코드가 없음.');
            callback(null, null);
        }
    })*/
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Overwriting ./src/app8.js


In [32]:
URL = 'http://localhost:3000/process/login'
#response = requests.get(URL)
data = {'id': 'test01','password':'123456'}
res = requests.post(URL, data=data)
res.text

'<h1>사용자 로그인 성공</h1><div><p>사용자 : donggyu</p></div><br><br><a href="/public/login.html">다시 로그인하기</a>'

In [35]:
URL = 'http://localhost:3000/process/adduser'
#response = requests.get(URL)
data = {'id': 'test02','password':'123456','name':'donggyu'}
res = requests.post(URL, data=data)
res.text

'<h1>사용자 추가 성공</h1><div><p>사용자 : donggyu</p></div><br><br><a href="/public/login.html">다시 로그인하기</a>'

### 데이터베이스의 이벤트와 스키마 타입
* String, Number, Boolean, Array, Buffer, Date, Objectid, Mixed
* type, required, unique, index
* find(), save(), update(), remove()
* static(), method()

# 인덱스와 메소드 사용하기

In [40]:
%%writefile ./src/app9.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;
var UserSchema;
var UserModel

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017/local';
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        
            
        UserSchema = mongoose.Schema({
            id: {type:String, required:true, unique:true},
            password: {type:String, required:true},
            name: {type:String, index:'hashed'},
            age: {type:Number, 'default':-1},
            created_at: {type:Date,index:{unique:false},'default':Date.now()},
            updated_at:{type:Date,index:{unique:false},'default':Date.now()}
        });
        console.log('UserSchema 정의함.');
        
        //함수 등록(이후 모델 객체에서 사용가능)
        UserSchema.static('findById', function(id, callback) {
            return this.find({id:id}, callback);        
        });
            
        /*UserSchema.statics.findById = function(id, callback) {
            return this.find({id:id}, callback);
        }*/
            
        UserSchema.static('findAll', function(callback){
            return this.find({}, callback);        
        })
            
        UserModel = mongoose.model('users2', UserSchema);
        console.log('UserModel 정의함.');
        
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/adduser').post(function(req,res){
    console.log('/process/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
})

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

router.route('/process/listuser').get(function(req,res){
    console.log('/process/listuser 라우팅 함수 호출됨.');
    if(database) {
        UserModel.findAll(function(err, results){
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
            if(results){
                console.dir(results);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write("<h3>사용자 리스트</h3>");
                res.write("<div><ul>");
                for(var i=0; i<results.length; i++){
                    var curId = results[i]._doc.id;
                    var curName = results[i]._doc.name;
                    res.write("    <li>#" + i + "-> " + curId + ", " + curName +
                             "</li>");
                }
                res.write("</ul></div>");
                res.end();
            }else {
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>조회된 사용자 없음.</h1>');
                res.end();
                return;
            }
        })
    }else{
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결안됨.</h1>');
        res.end();
        return;
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    UserModel.findById(id, function(err, result) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('아이디 %s로 검색됨.');
        if(result.length > 0){
            if(result[0]._doc.password === password) {
                console.log('비밀번호 일치함.');
                callback(null, result);
            } else {
                console.log('비밀번호 일치하지 않음.');
                callback(null, null);
            }
        }else{
            console.log('아이디 일치하는 사용자 없음.');
            callback(null, null);
        }
    });
    
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Overwriting ./src/app9.js


In [36]:
%%writefile ./src/public/listuser.html
<!DOCTYPE html>
<html>
    <head>
        <meta charset="utf-8">
        <title>사용자 리스트</title>
    </head>
    <body>
        <h1>사용자 리스트</h1>
        <br>
        <form method="get" action="/process/listuser">
            <table>
                <tr>
                    <td><label>아래 [전송] 버튼을 누르세요.</label></td>
                </tr>
            </table>
            <input type="submit" value="전송" name="">
        </form>
    </body>
</html>

Writing ./src/public/listuser.html


In [42]:
URL = 'http://localhost:3000/process/listuser'
response = requests.get(URL)
#data = {'id': 'test02','password':'123456','name':'donggyu'}
#res = requests.post(URL, data=data)
response.text

'<h3>사용자 리스트</h3><div><ul>    <li>#0-> test, 테스트</li></ul></div>'

# 비밀번호 암호화하여 저장하기

* virtual() 함수를 사용하면 문서 객체에 저장되는 속성이 아닌 가상 속성을 지정할 수 있음
* 문서 객체를 저장할 때 set(), 조회할 때 get() 메소드 사용

## virtual 사용법

In [43]:
%%writefile ./src/virtual_test.js
var mongoose = require('mongoose');

var database;
var UserSchema;
var UserModel;

function connectDB() {
    var databaseUrl = "mongodb://localhost:27017/local";
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        
        createUserSchema();    
        
        doTest();
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
}

function createUserSchema() {
    UserSchema = mongoose.Schema({
        id: {type:String, required:true, unique:true},
        name: {type:String, index:'hashed'},
        age: {type:Number, 'default':-1},
        created_at: {type:Date,index:{unique:false},'default':Date.now()},
        updated_at:{type:Date,index:{unique:false},'default':Date.now()}
    });
    console.log('UserSchema 정의함.');

    UserSchema.virtual('info')
    .set(function(info) {
        var splitted = info.split(' ');
        this.id = splitted[0];
        this.name = splitted[1];
        console.log('virtual info 속성 설정됨 : ' + this.id + ', ' + this.name);
    })
    .get(function() {return this.id + ' ' + this.name});
    
    UserModel = mongoose.model("users4", UserSchema);
    console.log("UserModel 정의함");
}

function doTest() {
    var user = new UserModel({"info":"test01 donggyu"});
    user.save(function(err){
        if(err) throw err;
        console.log('데이터 추가함.');
    })
}

connectDB();

Writing ./src/virtual_test.js


## 비밀번호 암호화 후 저장하기

In [45]:
%%writefile ./src/app10.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// 암호화 모듈
var crypto = require('crypto');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;
var UserSchema;
var UserModel

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017/local';
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
        
            
        UserSchema = mongoose.Schema({
            id: {type:String, required:true, unique:true,'default':''},
            hashed_password: {type:String, required:true,'default':''},
            salt:{type:String,required:true},
            name: {type:String, index:'hashed','default':''},
            age: {type:Number, 'default':-1},
            created_at: {type:Date,index:{unique:false},'default':Date.now()},
            updated_at:{type:Date,index:{unique:false},'default':Date.now()}
        });
        console.log('UserSchema 정의함.');
        
        UserSchema
            .virtual('password')
            .set(function(password) {
                this.salt = this.makeSalt();
                this.hashed_password = this.encryptPassword(password);
                console.log('virtual password 저장됨 : ' + this.hashed_pasword);
            })
            //.get()
            
        UserSchema.method('encryptPassword', function(plainText, inSalt){
            if(inSalt){
                return crypto.createHmac('sha1',inSalt).update(plainText).digest('hex');
            } else {
                return crypto.createHmac('sha1', this.salt).update(plainText).digest('hex');
            }
        })
        
        UserSchema.method('makeSalt', function() {
            return Math.round((new Date().valueOf() * Math.random())) + '';
        });
        
        UserSchema.method('authenticate', function(plainText,inSalt,hashed_password) {
            if(inSalt) //salt가 넘어온 경우
            {
                console.log('authenticate 호출됨.');
                return this.encryptPassword(plainText, inSalt) === hashed_password;
            } else {
                console.log('authenticate 호출됨.');
                return this.encryptPassword(plainText) === hashed_password;
            }
        })
            
        //함수 등록(이후 모델 객체에서 사용가능)
        UserSchema.static('findById', function(id, callback) {
            return this.find({id:id}, callback);        
        });
            
        /*UserSchema.statics.findById = function(id, callback) {
            return this.find({id:id}, callback);
        }*/
            
        UserSchema.static('findAll', function(callback){
            return this.find({}, callback);        
        })
            
        UserModel = mongoose.model('users3', UserSchema);
        console.log('UserModel 정의함.');
        
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/adduser').post(function(req,res){
    console.log('/process/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
})

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

router.route('/process/listuser').get(function(req,res){
    console.log('/process/listuser 라우팅 함수 호출됨.');
    if(database) {
        UserModel.findAll(function(err, results){
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
            if(results){
                console.dir(results);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write("<h3>사용자 리스트</h3>");
                res.write("<div><ul>");
                for(var i=0; i<results.length; i++){
                    var curId = results[i]._doc.id;
                    var curName = results[i]._doc.name;
                    res.write("    <li>#" + i + "-> " + curId + ", " + curName +
                             "</li>");
                }
                res.write("</ul></div>");
                res.end();
            }else {
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>조회된 사용자 없음.</h1>');
                res.end();
                return;
            }
        })
    }else{
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결안됨.</h1>');
        res.end();
        return;
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    UserModel.findById(id, function(err, result) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('아이디 %s로 검색됨.');
        if(result.length > 0){
            var user = new UserModel({id:id})
            var authenticated = user.authenticate(password, result[0],_doc.salt, result[0]._doc.hashed_password);
                
            if(authenticated) {
                console.log('비밀번호 일치함.');
                callback(null, result);
            } else {
                console.log('비밀번호 일치하지 않음.');
                callback(null, null);
            }
        }else{
            console.log('아이디 일치하는 사용자 없음.');
            callback(null, null);
        }
    });
    
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Overwriting ./src/app10.js


In [46]:
URL = 'http://localhost:3000/process/adduser'
#response = requests.get(URL)
data = {'id': 'test02','password':'123456','name':'donggyu'}
res = requests.post(URL, data=data)
res.text

'<h1>사용자 추가 성공</h1><div><p>사용자 : donggyu</p></div><br><br><a href="/public/login.html">다시 로그인하기</a>'

```
D:\폴더\학교\졸프\evote-project\WebServer\Study\src>node app10.js
익스프레스로 웹 서버를 실행함 : 3000
(node:12196) DeprecationWarning: current URL string parser is deprecated, and will be removed in a future version. To use the new parser, pass option { useNewUrlParser: true } to MongoClient.connect.
(node:12196) DeprecationWarning: current Server Discovery and Monitoring engine is deprecated, and will be removed in a future version. To use the new Server Discover and Monitoring engine, pass option { useUnifiedTopology: true } to the MongoClient constructor.
데이터베이스에 연결됨 : mongodb://localhost:27017/local
UserSchema 정의함.
UserModel 정의함.
(node:12196) DeprecationWarning: collection.ensureIndex is deprecated. Use createIndexes instead.
/process/adduser 라우팅 함수 호출됨.
요청 파라미터 : test02, 123456, donggyu
addUuser 호출됨 : test02, 123456, donggyu
virtual password 저장됨 : undefined
사용자 데이터 추가함.
model {
  '$__':
   InternalCache {
     strictMode: true,
     selected: undefined,
     shardval: undefined,
     saveError: undefined,
     validationError: undefined,
     adhocPaths: undefined,
     removing: undefined,
     inserting: true,
     saving: undefined,
     version: undefined,
     getters: {},
     _id: ObjectID { _bsontype: 'ObjectID', id: [Buffer] },
     populate: undefined,
     populated: undefined,
     wasPopulated: false,
     scope: undefined,
     activePaths:
      StateMachine {
        paths: [Object],
        states: [Object],
        stateNames: [Array],
        map: [Function] },
     pathsToScopes: {},
     cachedRequired: {},
     session: null,
     '$setCalled': Set { 'id', 'password', 'salt', 'hashed_password', 'name' },
     ownerDocument: undefined,
     fullPath: undefined,
     emitter:
      EventEmitter {
        _events: [Object: null prototype] {},
        _eventsCount: 0,
        _maxListeners: 0 },
     '$options': {},
     validating: null },
  isNew: false,
  errors: undefined,
  '$locals': {},
  '$op': null,
  _doc:
   { id: 'test02',
     hashed_password: '0e58419acfee337a9c2c50f2c441ca0d55ea25b1',
     name: 'donggyu',
     age: -1,
     created_at: 2020-06-12T22:37:37.316Z,
     updated_at: 2020-06-12T22:37:37.316Z,
     _id: ObjectID { _bsontype: 'ObjectID', id: [Buffer] },
     salt: '28454334176',
     __v: 0 } }
```

# MySQL 데이터베이스 사용하기(생략)
# 끝